In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/users/fyx/.local/python3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
train_pack = mz.datasets.wiki_qa.load_data('train', task='ranking')
valid_pack = mz.datasets.wiki_qa.load_data('dev', task='ranking', filter=True)
predict_pack = mz.datasets.wiki_qa.load_data('test', task='ranking', filter=True)

In [3]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, fixed_length_right=100, remove_stop_words=True)
train_pack_processed = preprocessor.fit_transform(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8023.59it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4625.69it/s]
Building VocabularyUnit from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 2718062.39it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2118/2118 [00:00<00:00, 8539.99it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 18841/18841 [00:04<00:00, 4597.65it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 78673.92it/s]


In [4]:
valid_pack_processed = preprocessor.transform(valid_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 122/122 [00:00<00:00, 8291.82it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 1115/1115 [00:00<00:00, 4586.15it/s]
Processing text_right with transform: 100%|██████████| 1115/1115 [00:00<00:00, 60616.83it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 237/237 [00:00<00:00, 6939.22it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit: 100%|██████████| 2300/2300 [00:00<00:00, 4599.53it/s]
Processing text_right with transform: 100%|██████████| 2300/2300 [00:00<00:00, 13918.50it/s]


In [5]:
ranking_task = mz.tasks.Ranking(loss=mz.losses.RankCrossEntropyLoss(num_neg=4))
ranking_task.metrics = [
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5),
    mz.metrics.MeanAveragePrecision()
]

In [6]:
bin_size = 30
model = mz.models.DRMM()
model.params['input_shapes'] = [[10,], [10, bin_size,]]
model.params['task'] = ranking_task
model.params['mask_value'] = 0
model.params['embedding_input_dim'] = preprocessor.context['vocab_size'] + 1
model.params['embedding_output_dim'] = 300
model.params['mlp_num_layers'] = 1
model.params['mlp_num_units'] = 10
model.params['mlp_num_fan_out'] = 1
model.params['mlp_activation_func'] = 'tanh'
model.params['optimizer'] = 'adadelta'
model.guess_and_fill_missing_params()
model.build()
model.compile()
model.backend.summary()

Parameter "name" set to DRMM.
Parameter "embedding_trainable" set to True.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 10, 300)      5002200     text_left[0][0]                  
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 10, 1)        300         embedding[0][0]                  
__________________________________________________________________________________________________
match_histogram (InputLayer)    (None, 10, 30)       0                                            
__________________________________

In [7]:
glove_embedding = mz.datasets.embeddings.load_glove_embedding(dimension=300)
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [8]:
# normalize the word embedding for fast histogram generating.
l2_norm = np.sqrt((embedding_matrix*embedding_matrix).sum(axis=1))
embedding_matrix = embedding_matrix / l2_norm[:, np.newaxis]

In [9]:
model.load_embedding_matrix(embedding_matrix)

In [10]:
pred_generator = mz.HistogramDataGenerator(data_pack=predict_pack_processed,
                                           embedding_matrix=embedding_matrix,
                                           bin_size=bin_size, 
                                           hist_mode='LCH')
pred_x, pred_y = pred_generator[:]
evaluate = mz.callbacks.EvaluateAllMetrics(model, 
                                           x=pred_x, 
                                           y=pred_y, 
                                           once_every=1, 
                                           batch_size=len(pred_y),
                                           model_save_path='./drmm_pretrained_model/'
                                          )

In [11]:
train_generator = mz.HistogramPairDataGenerator(train_pack_processed, embedding_matrix, bin_size, 'LCH',
                                                num_dup=2, num_neg=4, batch_size=20)
len(train_generator)

102

In [13]:
history = model.fit_generator(train_generator, epochs=30, callbacks=[evaluate], workers=30, use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 7s 73ms/step - loss: 1.5397
Validation: normalized_discounted_cumulative_gain@3(0): 0.5010246328297064 - normalized_discounted_cumulative_gain@5(0): 0.5683055337819536 - mean_average_precision(0): 0.5208140957157203
Epoch 2/30
102/102 [==============================] - 7s 64ms/step - loss: 1.4268
Validation: normalized_discounted_cumulative_gain@3(0): 0.5671621422842209 - normalized_discounted_cumulative_gain@5(0): 0.6271033747133542 - mean_average_precision(0): 0.5831514490551901
Epoch 3/30
102/102 [==============================] - 7s 66ms/step - loss: 1.3273
Epoch 3/30Validation: normalized_discounted_cumulative_gain@3(0): 0.5671621422842209 - normalized_discounted_cumulative_gain@5(0): 0.6271033747133542 - mean_average_precision(0): 0.5831514490551901
Validation: normalized_discounted_cumulative_gain@3(0): 0.5607839376465166 - normalized_discounted_cumulative_gain@5(0): 0.6168473766385947 - mean_average_precision(0): 0.576347675

Epoch 26/30
102/102 [==============================] - 7s 70ms/step - loss: 0.6131
Validation: normalized_discounted_cumulative_gain@3(0): 0.6134606290020046 - normalized_discounted_cumulative_gain@5(0): 0.6655056275779849 - mean_average_precision(0): 0.6275190536841742
Epoch 27/30
102/102 [==============================] - 7s 67ms/step - loss: 0.6100
Validation: normalized_discounted_cumulative_gain@3(0): 0.6104597472955772 - normalized_discounted_cumulative_gain@5(0): 0.6624432622048022 - mean_average_precision(0): 0.6253933001906411
Epoch 28/30
102/102 [==============================] - 7s 66ms/step - loss: 0.6066
Validation: normalized_discounted_cumulative_gain@3(0): 0.608574083035974 - normalized_discounted_cumulative_gain@5(0): 0.6632721865039131 - mean_average_precision(0): 0.6244271676025506
Epoch 29/30
102/102 [==============================] - 7s 68ms/step - loss: 0.6044
Validation: normalized_discounted_cumulative_gain@3(0): 0.608937862210369 - normalized_discounted_cumulat

In [14]:
drmm_model = mz.load_model('./drmm_pretrained_model/16')
test_generator = mz.HistogramDataGenerator(data_pack=valid_pack_processed[:10],
                                           embedding_matrix=embedding_matrix,
                                           bin_size=bin_size, 
                                           hist_mode='LCH')
test_x, test_y = test_generator[:]
prediction = drmm_model.predict(test_x)
prediction

array([[ 1.2173042 ],
       [-2.3363535 ],
       [ 0.2898716 ],
       [ 1.160675  ],
       [-1.8863118 ],
       [-2.571641  ],
       [ 2.253562  ],
       [ 2.3714523 ],
       [ 0.7752241 ],
       [ 0.05869245]], dtype=float32)

In [15]:
import shutil
shutil.rmtree('./drmm_pretrained_model/')